In [1]:
import torch
import torchvision
from torchvision import datasets

In [2]:
data=datasets.MNIST(root='./data',train=True,download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 16102789.33it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 435284.27it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4147264.63it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 11891715.84it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
X=data.data

In [4]:
X.shape,type(X)

(torch.Size([60000, 28, 28]), torch.Tensor)

In [5]:
y=data.targets

In [6]:
y.shape,type(y)

(torch.Size([60000]), torch.Tensor)

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
import torchvision.transforms as transforms

# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts the image to a PyTorch tensor and scales it to [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the tensor
])


In [9]:
X_train.shape,X_test.shape

(torch.Size([48000, 28, 28]), torch.Size([12000, 28, 28]))

In [18]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,X,y,transforms=None):
        self.X=X
        self.y=y
        self.transforms=transforms

    def __len__(self):
        return len(self.X)

    def __getitem__(self,idx):
        data=self.X[idx][None,...].float()/255
        label=self.y[idx]

        if self.transforms:
            data=self.transforms(data)

        return data,label

In [19]:
train_loader=torch.utils.data.DataLoader(CustomDataset(X_train,y_train),batch_size=32,shuffle=True)
test_loader=torch.utils.data.DataLoader(CustomDataset(X_test,y_test),batch_size=32,shuffle=False)

In [20]:
next(iter(train_loader))

[tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         ...,
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ..

In [26]:
class MyModel(torch.nn.Module):
  def __init__(self):
    super(MyModel,self).__init__()
    self.conv1=torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1)
    self.conv2=torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1)
    self.pool=torch.nn.MaxPool2d(kernel_size=2,stride=2)

    self.fc1=torch.nn.Linear(64*7*7,128)
    self.fc2=torch.nn.Linear(128,10)

  def forward(self,x):
    x=self.conv1(x)
    x=torch.nn.functional.relu(x)
    x=self.pool(x)
    x=self.conv2(x)
    x=torch.nn.functional.relu(x)
    x=self.pool(x)
    x=torch.flatten(x,1)
    x=self.fc1(x)
    x=torch.nn.functional.relu(x)
    x=self.fc2(x)
    return x

In [27]:
model=MyModel()
model(torch.rand(1,1,28,28)).shape

torch.Size([1, 10])

In [28]:
# model(torch.rand(1,28,28)).view(1,-1).shape

In [31]:
def train(model,train_loader,val_loader,optimizer,criterion,num_epochs):

  for epoch in range(num_epochs):
    total_train_loss=0
    total_val_loss=0
    for batch_idx,(data,target) in enumerate(train_loader):
      model.train()
      optimizer.zero_grad()
      output=model(data)
      loss=criterion(output,target)
      total_train_loss+=loss.item()
      loss.backward()
      optimizer.step()
    with torch.no_grad():
      for batch_idx,(data,target) in enumerate(val_loader):
        model.eval()
        output=model(data)
        val_loss=criterion(output,target)
        total_val_loss+=val_loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {total_train_loss/len(data):.4f}, Val Loss: {total_val_loss/len(data):.4f}')

In [33]:
criterion=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
train(model,train_loader,test_loader,optimizer,criterion,num_epochs=10)

Epoch 1/10, Train Loss: 2.4718, Val Loss: 0.5762
Epoch 2/10, Train Loss: 1.5583, Val Loss: 0.5184
Epoch 3/10, Train Loss: 1.1267, Val Loss: 0.4170
Epoch 4/10, Train Loss: 0.7884, Val Loss: 0.4651
Epoch 5/10, Train Loss: 0.6870, Val Loss: 0.5202
Epoch 6/10, Train Loss: 0.5282, Val Loss: 0.5035
Epoch 7/10, Train Loss: 0.4505, Val Loss: 0.5869
Epoch 8/10, Train Loss: 0.2631, Val Loss: 0.5803
Epoch 9/10, Train Loss: 0.3997, Val Loss: 0.6288
Epoch 10/10, Train Loss: 0.2522, Val Loss: 0.6375
